## Cálculo de NDVI usando imágenes de planet

In [1]:
import rasterio
import pandas as pd
import geopandas as gpd

In [2]:
def calculate_ndvi(image_file, output_ndvi_file, cmap_nvdi_image):
    """
    Calculate the NDVI for a given image.
    
    Parameters:
    - image_file (str): Path to the image file.
    - output_ndvi_file (str): Path to save the output NDVI file. If the file already exists, it will be overwritten.
    - cmap_nvdi_image (str): Path to save the colormap NDVI image.
    
    Returns:
    None
    """

    # Load red and NIR bands - note all PlanetScope 4-band images have band order BGRN
    with rasterio.open(image_file) as src:
        band_red = src.read(3)
        band_nir = src.read(4)

    # Calculate NDVI while avoiding 0/0 situation
    denominator = band_nir + band_red + 1e-10
    ndvi = (band_nir.astype(float) - band_red.astype(float)) / denominator

    # Clip NDVI to the range [-1, 1]
    ndvi = np.clip(ndvi, -1, 1)

    # Convert NaN values to a chosen value (0 in this case)
    ndvi = np.nan_to_num(ndvi)

    # Normalize NDVI to [0, 1] for visualization
    ndvi_normalized = (ndvi + 1) / 2.0

    # Set spatial characteristics of the output object to mirror the input
    kwargs = src.meta
    kwargs.update(
        dtype=rasterio.float32,
        count = 1)

    # Check if the output raster already exists, if not, create it
    if os.path.exists(output_ndvi_file):
        os.remove(output_ndvi_file)

    # Create (or overwrite) the NDVI file
    with rasterio.open(output_ndvi_file, 'w', **kwargs) as dst:
        dst.write_band(1, ndvi.astype(rasterio.float32))
        
    # Save the normalized NDVI as a colormap image
    plt.imsave(cmap_nvdi_image, ndvi_normalized, cmap=plt.cm.RdYlGn)

In [232]:
for i in files: 
    
    if i.endswith('.tiff'):
        ndvi_path = i
        output_ndvi_file = i.replace('.tiff', '_ndvi.tiff')
        nvdi_png_image = i.replace('.tiff', '_ndvi.png')
        calculate_ndvi(ndvi_path, output_ndvi_file, nvdi_png_image)
    else: 
        continue

In [233]:
files = get_files_in_folder(folder)

In [294]:
def filter_by_substring(elements, target_string):
    """
    Filters elements of a list based on the presence of a target substring.
    
    Parameters:
    - elements (list): List of elements to be filtered.
    - target_string (str): Substring used for filtering.
    
    Returns:
    - list: Elements of the input list that contain the target substring.
    """
    return [element for element in elements if target_string in element]


def sort_by_endings(files, endings):
    """
    Sort the 'files' list based on the order of 'endings'.
    
    Parameters:
    - files (list): List of file names.
    - endings (list): List of endings to sort by.
    
    Returns:
    List: Sorted list of file names.
    """
    return sorted(files, key=lambda x: [x.endswith(ending) for ending in endings].index(True))

def get_image_files_project(project, files):
    files_1 = filter_by_substring(files, project)
    images = filter_by_substring(files_1, '.png')
    endings = ['fin_ndvi.png', 'fin.png', 'inicio_ndvi.png', 'inicio.png']
    
    return sort_by_endings(images, endings)

def get_titles_from_images(images_files, project):
    titles = []
    for i in images_files: 
        titles = titles + [extract_after_word(i, project+'_', 7)]
    return titles


def create_mosaic(png_files, titles, mosaic_title, note, output_file):
    """
    Create a 2x2 mosaic of 4 PNG images with titles and save as a new PNG.

    Parameters:
    - png_files (list): List of 4 paths to PNG files.
    - titles (list): List of 4 titles for each PNG.
    - mosaic_title (str): Title for the mosaic.
    - note (str): Note to be added at the end of the mosaic.
    - output_file (str): Path to save the output mosaic PNG.
    
    Returns:
    None
    """

    if len(png_files) != 4 or len(titles) != 4:
        raise ValueError("Please provide exactly 4 PNG files and 4 titles.")

    fig, axs = plt.subplots(2, 2, figsize=(10, 10))  # Adjust size as needed

    for ax, png_file, title in zip(axs.ravel(), png_files, titles):
        img = plt.imread(png_file)
        ax.imshow(img)
        ax.set_title(title)
        ax.axis('off')

    plt.suptitle(mosaic_title)
    fig.subplots_adjust(bottom=0.15)
    fig.text(0.5, 0.08, note, ha='center')

    plt.savefig(output_file, dpi=300)  # Adjust dpi as needed
    plt.close()  # Close the plot to prevent it from being shown in the notebook


def get_mosaic_file(project, files, lista_proyectos):
    
    images = get_image_files_project(project, files)
    
    titles = get_titles_from_images(images, project)
    
    out_file_path = 'out/{}.png'.format(project)
    proyectos = ', '.join(map(str, lista_proyectos))
    
    create_mosaic(images, 
              titles, 
              'Análisis del punto ' + project, 
              'Proyectos que operaron en el municipo: ' + proyectos,
              out_file_path)
    

In [300]:
for index, row in gdf.iterrows():
    get_mosaic_file(row['id'], files, row['proyectos_activos'])

In [396]:
def get_nvdi_raster_files_project(files, project):
    files_1 = filter_by_substring(files, project)
    rasters = filter_by_substring(files_1, '.tiff')
    endings = ['inicio_ndvi.tiff', 'fin_ndvi.tiff', 'fin.tiff',  'inicio.tiff']
    rasters = sort_by_endings(rasters, endings)
    return rasters[0:2]

In [399]:
gdf['ndvi_paths'] = gdf.apply(lambda x: get_nvdi_raster_files_project(files, x['id']), axis=1)
gdf['ndvi_inicio_path'] = gdf['ndvi_paths'].apply(lambda x: x[0])
gdf['ndvi_final_path'] = gdf['ndvi_paths'].apply(lambda x: x[1])

In [403]:
def calculate_mean_ignoring_zeros(raster_path):
    """
    Calculate the mean value of a raster after setting all 0 values to NaN.

    Parameters:
    - raster_path: path to the raster file.

    Returns:
    - Mean value of the raster ignoring 0 values.
    """
    # Read the raster
    with rasterio.open(raster_path) as src:
        raster_data = src.read(1)  # Reading the first band
        
        # Convert zeros to NaN
        raster_data[raster_data == 0] = np.nan
        
        # Calculate the mean while ignoring NaN values
        mean_val = np.nanmean(raster_data)
    
    return mean_val


In [406]:
gdf['ndvi_inicio'] = gdf['ndvi_inicio_path'].apply(calculate_mean_ignoring_zeros)
gdf['ndvi_final'] = gdf['ndvi_final_path'].apply(calculate_mean_ignoring_zeros)

In [410]:
df1 = gdf[['proyecto_revisar', 'id', 'ndvi_inicio', 'ndvi_final']]